In [ ]:
import urllib.request
import xarray as xr
from xrspatial import ndvi
import datashader as ds
from datashader.transfer_functions import shade
from datashader import transfer_functions as tf
from colorcet import palette
import json
import matplotlib.pyplot as plt
import holoviews as hv
from holoviews import opts
hv.extension('bokeh', width=80)
from holoviews.operation.datashader import regrid, shade
import numpy as np
def load_scale_factors(filename, band_number):
    with open(filename) as f:
        metadata = json.load(f)
    M_p = metadata['L1_METADATA_FILE'] \
                  ['RADIOMETRIC_RESCALING'] \
                  ['REFLECTANCE_MULT_BAND_{}'.format(band_number)]
    A_p = metadata['L1_METADATA_FILE'] \
                  ['RADIOMETRIC_RESCALING'] \
                  ['REFLECTANCE_ADD_BAND_{}'.format(band_number)]
    return M_p, A_p
def calculate_reflectance(ds, band_number, metafile='data/LC08_L1TP_042034_20170616_20170629_01_T1_MTL.json'):
    M_p, A_p = load_scale_factors(metafile, band_number)
    toa = M_p * ds + A_p
    return toa

In [ ]:
url = 'http://landsat-pds.s3.amazonaws.com/c1/L8/042/034/LC08_L1TP_042034_20170616_20170629_01_T1/'
redband = 'LC08_L1TP_042034_20170616_20170629_01_T1_B{}.TIF'.format(4)
nirband = 'LC08_L1TP_042034_20170616_20170629_01_T1_B{}.TIF'.format(5)
mtlfile = 'LC08_L1TP_042034_20170616_20170629_01_T1_MTL.json'

In [ ]:
path2data = "data/"

In [ ]:
urllib.request.urlretrieve(url+redband, path2data+redband)

In [ ]:
urllib.request.urlretrieve(url+nirband, path2data+nirband)

In [ ]:
urllib.request.urlretrieve(url+mtlfile, path2data+mtlfile)

In [ ]:
red = xr.open_rasterio("data/LC08_L1TP_042034_20170616_20170629_01_T1_B4.TIF")
nir = xr.open_rasterio("data/LC08_L1TP_042034_20170616_20170629_01_T1_B5.TIF")
red_toa = calculate_reflectance(red, band_number=4)
nir_toa = calculate_reflectance(nir, band_number=5)
ndvi = (nir_toa - red_toa) / (nir_toa + red_toa)

In [ ]:
plt.figure()
im = ndvi.squeeze().compute().plot.imshow(cmap='BrBG', vmin=-0.5, vmax=1)
plt.axis('equal')
plt.show()

In [ ]:
opts.defaults(opts.RGB(width=600, height=600))

nodata= 1

def one_band(b):
    xs, ys = b['x'], b['y']
    b = ds.utils.orient_array(b)
    a = (np.where(np.logical_or(np.isnan(b),b<=nodata),0,255)).astype(np.uint8)    
    col, rows = b.shape
    return hv.RGB((xs, ys[::-1], b, b, b, a), vdims=list('RGBA'))

In [ ]:
shade(regrid(one_band(ndvi[0])), cmap=['black', 'white']).redim(x='Longitude', y='Latitude') + \
shade(regrid(one_band(ndvi[0])), cmap=['red', 'blue','green','white']).redim(x='Longitude', y='Latitude')

In [ ]:
# shade(ndvi[0], cmap=['black', 'white']).redim(x='Longitude', y='Latitude')
shade(one_band(ndvi[0])).redim(x='Longitude', y='Latitude')

In [ ]:
from xrspatial import hillshade

In [ ]:
from datashader.colors import Elevation
shade(regrid(one_band(ndvi[0])), cmap=Elevation).redim(x='Longitude', y='Latitude')

In [ ]:
hillshaded = hillshade(ndvi[0])

In [ ]:
shade(regrid(one_band(ndvi[0])), cmap=['white','black']).redim(x='Longitude', y='Latitude') +\
shade(regrid(one_band(hillshaded)), cmap=['white','black']).redim(x='Longitude', y='Latitude')

In [ ]:
from datashader.transfer_functions import stack
from datashader.transfer_functions import dynspread
from datashader.transfer_functions import set_background

In [ ]:
stack(shade(regrid(one_band(ndvi[0])), cmap=['gray', 'white'], alpha=255, how='linear'),
      shade(regrid(one_band(hillshaded)), cmap=Elevation, alpha=128, how='linear'))

In [ ]:
stack(shade(regrid(one_band(ndvi[0])), cmap=['gray', 'white'], alpha=255),shade(regrid(one_band(hillshaded)), cmap=Elevation, alpha=128))